### Imports

In [44]:
import pandas as pd
import ast
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm, tnrange
tqdm.pandas()
import dateutil
import datetime

#### Create Dataset

In [45]:
df = pd.read_csv("./tokens_data.csv",index_col=0)

In [46]:
values = ['director_anchors','genre','star_anchs','tokens']
for val in values:
  df[val]=df[val].apply(lambda x: ast.literal_eval(x) if type(x)==str else np.nan)

SyntaxError: invalid syntax (<unknown>, line 1)

In [4]:
df.head()

,name,profile,summary,director_anchors,release,genre,star_anchs,main_prod,summary_len,word_count,date,datetime,year,month,summary_tokens
0,Star Wars: Episode VII - The Force Awakens,/title/tt2488496/,"Three decades after the Empire's defeat, a new...",[/name/nm0009190/],18 December 2015,"[Action, Adventure, Sci-Fi]","[/name/nm5397459/, /name/nm3915784/, /name/nm1...",Lucasfilm,2,34,2015-12-18,2015-12-18,2015,12,"[2, 132, 3784, 75, 14, 1563, 22, 18, 2143, 15,..."
1,Avengers: Endgame,/title/tt4154796/,After the devastating events of Avengers: Infi...,"[/name/nm0751577/, /name/nm0751648/]",26 April 2019,"[Action, Adventure, Drama]","[/name/nm0000375/, /name/nm0262635/, /name/nm0...",Marvel Studios,2,37,2019-04-26,2019-04-26,2019,4,"[2, 75, 14, 16737, 963, 16, 17380, 45, 18425, ..."
2,Avatar,/title/tt0499549/,A paraplegic Marine dispatched to the moon Pan...,[/name/nm0000116/],18 December 2009,"[Action, Adventure, Fantasy]","[/name/nm0941777/, /name/nm0757855/, /name/nm0...",Twentieth Century Fox,1,27,2009-12-18,2009-12-18,2009,12,"[2, 21, 2258, 5106, 12964, 2061, 16520, 20, 14..."
3,Black Panther,/title/tt1825683/,"T'Challa, heir to the hidden but advanced king...",[/name/nm3363032/],16 February 2018,"[Action, Adventure, Sci-Fi]","[/name/nm1569276/, /name/nm0430107/, /name/nm2...",Marvel Studios,1,30,2018-02-16,2018-02-16,2018,2,"[2, 13, 38, 22, 10989, 531, 15, 7678, 20, 14, ..."
4,Avengers: Infinity War,/title/tt4154756/,The Avengers and their allies must be willing ...,"[/name/nm0751577/, /name/nm0751648/]",27 April 2018,"[Action, Adventure, Sci-Fi]","[/name/nm0000375/, /name/nm1165110/, /name/nm0...",Marvel Studios,1,32,2018-04-27,2018-04-27,2018,4,"[2, 14, 17380, 17, 66, 6255, 491, 44, 4452, 20..."


In [5]:
bag_of_words = []
for row in df.itertuples():
  result = ""
  result+=row.director_anchors[0].replace("/name/nm","").replace("/","")
  for val in row.genre:
    val=val.replace("-","")
    result+=" "+val.lower()+" "
  for val in row.star_anchs:
    result+=" "+val.replace("/name/nm","").replace("/","")+" "
  result+=row.summary.lower()
  bag_of_words.append(result)

In [6]:
df['bag_of_words']=bag_of_words

In [12]:
df=df.loc[df['year']>2010]

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
count_matrix.shape

(53685, 160897)

In [17]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

# # generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

#### Create Cosin Similarities

In [5]:
similarities = cosine_similarity(np.array(df['summary_tokens'].tolist(), dtype='float32'))

In [18]:
df=df.reset_index(drop=True)

In [19]:
#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(unique_id, cosine_sim = cosine_sim):

  # initializing the empty list of recommended movies
  recommended_movies = []

#     breakpoint()
  # gettin the index of the movie that matches the unique id
  idx = df[df.profile==unique_id].index[0]

  # creating a Series with the similarity scores in descending order
  score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

  # getting the indexes of the 10 most similar movies
  top_10_indexes = list(score_series.iloc[1:11].index)

  # populating the list with the titles of the best 10 matching movies
  for i in top_10_indexes:
    recommended_movies.append(list(df.index)[i])

  return recommended_movies

In [38]:
def print_movie(index):
  print(f"Selected Movie: {df.iloc[index]['name']}")
  values = recommendations(df.profile[index])
  for val in values:
    print(df.iloc[val]['name'])
    print(df.iloc[val].summary)
    print("=================")

In [43]:
print_movie(44)

Selected Movie: Guardians of the Galaxy
Golden State
A comedy about a group of Israelis who come to the United States in order to pursue the "American dream" and the ongoing challenges that each one of them must face while trying to manifest such dreams.
The Hit
A rare coin is the subject of international intrigue when a up and coming art curator must travel to the US to seek out the coin with the help of a sassy starlet.
Thor: Ragnarok
Imprisoned on the planet Sakaar, Thor must race against time to return to Asgard and stop Ragnarök, the destruction of his world, at the hands of the powerful and ruthless villain Hela.
Río de oro
Sonora, Mexico, 1852. During war with the apaches and the invasion of the US in Mexico, a muleteer decides to leave home to find a better place to live. To do so he will have to cross the land of the Chiricahuas to find gold.
Spy Kids 4: All the Time in the World
A retired spy is called back into action, and to bond with her new step-children, she invites them

In [34]:
for val in values:
  print(df.iloc[val]['name'])
  print(df.iloc[val].summary)
  print("=================")

Heaven: The Adventure Begins
Set between the New World (Heaven) and the Old World (Earth) the story follows the life and afterlife of Jonathan Stone and all of those lives around him from his past, present and future.
Bugged
Enter a micro world of colors and fun! Join butterflies, caterpillars, spiders and all the amazing creatures that inhabit the marsh. When a new frog shows up in the swamp the bugs welcome ...
Pete Winning and the Pirates
Armed with laser guns and wit, Pete Winning and his rag-tag crew navigate a flooded earth in the not-too-distant future. Braving mutants, pirates and the seas, they search for the self-crowned evil Queen's coveted maps of the new world.
Quackerz
The battle between ducks and humans for the place under the sun.
Dog Geniuses
Slobberstick and Mugs are two streetwise dogs from the wrong side of the tracks, whose tails wag for knowledge and ears perk for facts! And when the local canine university mistakes the ...
Thomas & Friends: Journey Beyond Sodor
J

In [10]:
df.iloc[values]

,name,profile,summary,director_anchors,release,genre,star_anchs,main_prod,summary_len,word_count,date,datetime,year,month,summary_tokens
35185,Speechless,/title/tt11106572/,Jonah suffers from an anxiety disorder which c...,[/name/nm4604171/],20 November 2019,"[Comedy, Drama, Family]","[/name/nm11057944/, /name/nm2485463/, /name/nm...",Watershed Motion Pictures,2,33,2019-11-20,2019-11-20,2019,11,"[2, 17800, 6586, 18, 37, 40, 10248, 7942, 56, ..."
48186,Public Access,/title/tt0107895/,Brewster seems to be an almost too perfect exa...,[/name/nm0001741/],January 1993,"[Drama, Thriller]","[/name/nm0549825/, /name/nm0930095/, /name/nm0...",Cinemabeam,2,32,1993-01-17,1993-01-17,1993,1,"[2, 24987, 2206, 20, 44, 40, 557, 266, 2107, 8..."
41655,Sci-Fighter,/title/tt0390463/,A computer virus has invaded a virtual reality...,[/name/nm0131064/],17 May 2005,"[Action, Sci-Fi]","[/name/nm0933310/, /name/nm0001686/, /name/nm0...",Gorilla Pictures,2,41,2005-05-17,2005-05-17,2005,5,"[2, 21, 1428, 6231, 63, 11469, 21, 6599, 2902,..."
33402,On the Market,/title/tt1500845/,"Megan Brothers has it all: A wonderful family,...",[/name/nm2728891/],2009,"[Comedy, Romance]","[/name/nm3220632/, /name/nm2893045/, /name/nm3...",Southern Horizon Pictures,2,33,2009-01-17,2009-01-17,2009,1,"[2, 13972, 1670, 63, 32, 65, 45, 21, 5934, 190..."
31514,Hollywood Whores,/title/tt1709118/,"Struggling artists in Los Angeles, from studio...",[/name/nm2080488/],20 August 2011,"[Comedy, Drama, Music]","[/name/nm0489196/, /name/nm2478830/, /name/nm5...",Palmarejo Films,2,27,2011-08-20,2011-08-20,2011,8,"[2, 7587, 1542, 19, 1213, 1558, 15, 37, 1120, ..."
39439,"Macbeth 3000: This Time, It's Personal",/title/tt0482006/,Macbeth and Macduff were agents of the dangero...,[/name/nm2054108/],15 July 2005,"[Action, Comedy]","[/name/nm2054495/, /name/nm2053062/, /name/nm2...",SuperGun Cinema,2,29,2005-07-15,2005-07-15,2005,7,"[2, 27580, 17, 1572, 43, 5386, 46, 5125, 16, 1..."
1872,The Three Stooges,/title/tt0383010/,While trying to save their childhood orphanage...,"[/name/nm0268370/, /name/nm0268380/]",13 April 2012,"[Comedy, Family]","[/name/nm0005003/, /name/nm0224565/, /name/nm0...",Twentieth Century Fox,1,26,2012-04-13,2012-04-13,2012,4,"[2, 133, 749, 20, 2079, 66, 4292, 19492, 15, 1..."
21294,The Final Project,/title/tt3758334/,Six eager students strike out to explore and r...,[/name/nm6462306/],4 March 2016,"[Horror, Thriller]","[/name/nm1967072/, /name/nm3259039/, /name/nm5...",3rd Fathom Entertainment,1,29,2016-03-04,2016-03-04,2016,3,"[2, 490, 8705, 573, 3333, 70, 20, 8594, 17, 57..."
18107,Gunfight at El Rio,/title/tt4648270/,El Rio is run by a corrupt oil baron who wants...,[/name/nm7293783/],15 May 2008,[Western],"[/name/nm4613501/, /name/nm4656026/, /name/nm5...",Thespis,2,41,2008-05-15,2008-05-15,2008,5,"[2, 931, 4017, 25, 485, 34, 21, 11305, 1585, 3..."
27561,Unexpected Race,/title/tt2166289/,A young woman must live with her father who sh...,[/name/nm4107137/],30 September 2003,[Drama],"[/name/nm2168769/, /name/nm3014457/, /name/nm2...",Wolf Productions UT,3,35,2003-09-30,2003-09-30,2003,9,"[2, 21, 461, 524, 491, 515, 29, 36, 321, 72, 3..."
